In [ ]:
import os
import h5py
import pickle
import numpy as np
from utils import LatLonStandardScaler, StandardScaler

from sgptools.kernels import *
from sgptools.utils.gpflow import *
from sgptools.utils.metrics import *
from sgptools.utils.misc import polygon2candidates

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

gpflow.config.set_default_float(np.float32)
gpflow.config.set_default_jitter(1e-1)

In [ ]:
def plot_results(X_candidates, y_candidates, var, 
                 X, waypoints,
                 title=None, fname=None, 
                 figsize=(10, 4), update_waypoint=None):
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=figsize)

    # Reconstruction
    map = ax[0].scatter(X_candidates[:, 1], X_candidates[:, 0],
                        c=y_candidates)
    ax[0].scatter(X[:, 1], X[:, 0], c='r', s=0.5, marker='.', label='Data')
    ax[0].plot(waypoints[:, 1], waypoints[:, 0], marker='o', c='r', label='Planned Path')
    if update_waypoint is not None:
        ax[0].scatter(waypoints[update_waypoint, 1], waypoints[update_waypoint, 0],
                      marker='o', c='g', label='Update Waypoint', zorder=10)
    ax[0].set_xlabel("X")
    ax[0].set_ylabel("Y")
    divider = make_axes_locatable(ax[0])
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(map, cax=cax, orientation='vertical')
    ax[0].set_title("Predicted Bathymetry")
    ax[0].set_aspect('equal')
    ax[0].legend()


    # Prediction Variance
    map = ax[1].scatter(X_candidates[:, 1], X_candidates[:, 0],
                        c=var)
    ax[1].set_xlabel("X")
    ax[1].set_yticks([])
    divider = make_axes_locatable(ax[1])
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(map, cax=cax, orientation='vertical')
    ax[1].set_title("Prediction Variance")
    ax[1].set_aspect('equal')

    if title is not None:
        plt.suptitle(title)
    if fname is not None:
        plt.savefig(fname, dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
mission_log = "../launch/data/Coverage-mission-2026-01-01-16-03-32"
num_samples = 5000

# data file
fname = os.path.join(mission_log,
                     "mission-log.hdf5")

# load data
with h5py.File(fname, "r") as f:
    print(f.keys())
    fence_vertices = f["fence_vertices"][:].astype(float)
    X = f["X"][:].astype(float)
    y = f["y"][:].astype(float)

    X_init = f["X_init"][:].astype(float)
    y_init = f["y_init"][:].astype(float)

    waypoints = {}
    for key in f.keys():
        if "path" in key:
            waypoints[key] = f[key][:].astype(float)

print(f'Mission Log: {mission_log}')
print(f'Number of data samples: {X.shape[0]}')
print(f'Number of reconstruction samples: {num_samples}')

In [ ]:
X_candidates = polygon2candidates(fence_vertices, 
                                  num_samples=num_samples)
X_scaler = LatLonStandardScaler()
X_scaler.fit(X_candidates)
X_scaler.scale_ *= 0.35
X_candidates = X_scaler.transform(X_candidates)
y_scaler = StandardScaler()
y_scaler.fit(y)

y = y_scaler.transform(y)
X = X_scaler.transform(X)

y_init = y_scaler.transform(y_init)
X_init = X_scaler.transform(X_init)

initial_path = X_scaler.transform(waypoints['initial_path-16-03-32'])
sol_path = X_scaler.transform(waypoints['coverage_path-16-12-30'])

In [ ]:
plt.scatter(X_init[:, 0], X_init[:, 1], s=20)
plt.plot(initial_path[:, 0], initial_path[:, 1], c='r')

In [ ]:
plt.scatter(X[:, 0], X[:, 1], s=20)
plt.plot(sol_path[:, 0], sol_path[:, 1], c='r')

In [ ]:
kernel = get_kernel('Attentive')([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

_, noise_variance, kernel, init_model = get_model_params(
    X_train=X_sol.astype(np.float32),
    y_train=X_sol.astype(np.float32),
    optimizer='tf.Nadam',
    kernel=kernel,
    return_model=True,
    verbose=True,
)

In [ ]:
y_candidates = kernel.get_lengthscales(X_candidates.astype(np.float32))
plt.scatter(X_candidates[:, 0], X_candidates[:, 1], s=1, c=y_candidates)

In [ ]:
candidates_y, var = gpr_gt.predict_f(X_candidates)
candidates_y = candidates_y.numpy()
var = var.numpy()

for key in waypoints.keys():
    update_waypoint = int(key.split('_')[1][:-9])-1
    update_waypoint = np.max([update_waypoint, 0])
    print(f"Update Waypoint: {update_waypoint}")
    plot_results(X_scaler.inverse_transform(X_candidates), 
                 y_scaler.inverse_transform(candidates_y), 
                 var, X_scaler.inverse_transform(X), waypoints[key], "",
                 os.path.join(mission_log, f"UpdateWaypoint{key.split('_')[1][:-9]}.png"),
                 update_waypoint=update_waypoint,
                 figsize=(18, 4))